# Gerar Bronze 

In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pyspark.pandas as ps
import seaborn as sns

from matplotlib import dates
from pyspark.sql.functions import col, to_date
from pyspark.sql.avro.functions import *
#
# Nome da aplicação Spark
#
APP_NAME="GerarBronze"
ps.options.display.max_rows = 10

/opt/conda/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
%run StartSpark.ipynb

In [3]:
%config SqlMagic.lazy_execution = True

In [4]:
%sql spark

In [5]:
schema = """
    {
        "type": "record",
        "name": "Stock",
        "fields": [
            {"name": "ticker", "type": "string"},
            {"name": "timestamp", "type": "string"},
            {"name": "open", "type": "double"},
            {"name": "high", "type": "double"},
            {"name": "low", "type": "double"},
            {"name": "close", "type": "double"},
            {"name": "volume", "type": "long"}
        ]
    }
"""

# FK

In [6]:
tabela = "s3a://nemesys-demo1/lakehouse/bronze/stocks_intraday"
checkpoint = "s3a://nemesys-demo1/lakehouse/bronze/checkpoint/stocks_intraday"
offset = "latest"

In [7]:
# (spark
#     .readStream
#     .format("kafka")
#     .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP)
#     .option("subscribe", KAFKA_TOPIC)
#     .option("startingOffsets", offset)
#     # .option("security.protocol", "SSL")
#     .load()
#     .select(from_avro(col("value"), schema).alias("value"))
#     .select("value.*")
#     .writeStream
#     .format('delta')
#     .outputMode('append')
#     .option('mergeSchema', 'true')
#     .option('checkpointLocation', checkpoint)
#     .trigger(once=True)
#     .start(tabela)
#     .awaitTermination()
# )    

In [12]:
%%time
df = (spark.read.format("delta").load(tabela))

CPU times: user 4.02 ms, sys: 0 ns, total: 4.02 ms
Wall time: 1.29 s


In [20]:
%%time
df.count()

CPU times: user 65.8 ms, sys: 15.2 ms, total: 81 ms
Wall time: 2min 46s


11982387

7.452.135
11.126.251
11.172.507
11.210.353
11.982.387

In [10]:
df.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: long (nullable = true)



In [14]:
%%time
(df
.withColumn("dia",to_date("timestamp"))
.groupBy("dia", "ticker")
.count()
.orderBy("dia", "ticker", ascending=False)
.show(n=100)
)

+----------+------+------+
|       dia|ticker| count|
+----------+------+------+
|2024-06-18|  TSLA| 13141|
|2024-06-18|    NU| 12765|
|2024-06-18|  MSFT| 12601|
|2024-06-18|  GOOG| 11869|
|2024-06-18|  AMZN| 13194|
|2024-06-18|  AAPL| 13176|
|2024-06-17|  TSLA|169398|
|2024-06-17|    NU|134679|
|2024-06-17|  MSFT|159473|
|2024-06-17|  GOOG|138308|
|2024-06-17|  AMZN|167867|
|2024-06-17|  AAPL|169343|
|2024-06-14|  TSLA|166311|
|2024-06-14|    NU|133578|
|2024-06-14|  MSFT|144866|
|2024-06-14|  GOOG|132321|
|2024-06-14|  AMZN|157500|
|2024-06-14|  AAPL|168288|
|2024-06-13|  TSLA|168139|
|2024-06-13|    NU|148260|
|2024-06-13|  MSFT|146963|
|2024-06-13|  GOOG|120088|
|2024-06-13|  AMZN|166279|
|2024-06-13|  AAPL|169379|
|2024-06-12|  TSLA|169413|
|2024-06-12|    NU|153559|
|2024-06-12|  MSFT|162654|
|2024-06-12|  GOOG|138339|
|2024-06-12|  AMZN|161484|
|2024-06-12|  AAPL|169299|
|2024-06-11|  TSLA|166108|
|2024-06-11|    NU|148419|
|2024-06-11|  MSFT|149585|
|2024-06-11|  GOOG|111929|
|

In [15]:
%%time
(df
.dropDuplicates()
.withColumn("dia",to_date("timestamp"))
.groupBy("dia", "ticker")
.count()
.orderBy("dia", "ticker", ascending=False)
.show(n=100)
)

+----------+------+-----+
|       dia|ticker|count|
+----------+------+-----+
|2024-06-18|  TSLA|  235|
|2024-06-18|    NU|  186|
|2024-06-18|  MSFT|  216|
|2024-06-18|  GOOG|  177|
|2024-06-18|  AMZN|  229|
|2024-06-18|  AAPL|  253|
|2024-06-17|  TSLA|  583|
|2024-06-17|    NU|  367|
|2024-06-17|  MSFT|  530|
|2024-06-17|  GOOG|  386|
|2024-06-17|  AMZN|  581|
|2024-06-17|  AAPL|  586|
|2024-06-14|  TSLA|  571|
|2024-06-14|    NU|  365|
|2024-06-14|  MSFT|  435|
|2024-06-14|  GOOG|  357|
|2024-06-14|  AMZN|  495|
|2024-06-14|  AAPL|  595|
|2024-06-13|  TSLA|  527|
|2024-06-13|    NU|  398|
|2024-06-13|  MSFT|  464|
|2024-06-13|  GOOG|  325|
|2024-06-13|  AMZN|  533|
|2024-06-13|  AAPL|  630|
|2024-06-12|  TSLA|  619|
|2024-06-12|    NU|  442|
|2024-06-12|  MSFT|  545|
|2024-06-12|  GOOG|  413|
|2024-06-12|  AMZN|  529|
|2024-06-12|  AAPL|  558|
|2024-06-11|  TSLA|  521|
|2024-06-11|    NU|  399|
|2024-06-11|  MSFT|  477|
|2024-06-11|  GOOG|  304|
|2024-06-11|  AMZN|  550|
|2024-06-11|